In [1]:
import numpy as np
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping

In [2]:
train_dir = r"E:\imgs\train"
test_dir = r"E:\imgs\test"

In [3]:
train_datagen = ImageDataGenerator(
      rescale = 1./255
)

test_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = train_datagen.flow_from_directory(
	train_dir,
	target_size=(32,32),
	class_mode='categorical',
    batch_size=100,
    color_mode='grayscale',
    subset='training'

)

validation_generator = train_datagen.flow_from_directory(
	train_dir,
	target_size=(32,32),
	class_mode='categorical',
    batch_size=100,
    color_mode='grayscale'
)

test_generator = test_datagen.flow_from_directory(
	test_dir,
	target_size=(32,32),
	class_mode='categorical',
    batch_size=100,
    color_mode='grayscale'
)

Found 22424 images belonging to 10 classes.
Found 22424 images belonging to 10 classes.
Found 79726 images belonging to 1 classes.


In [4]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(32,32,1)),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 64)        640       
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 64)        36928     
                                                                 
 max_pooling2d (MaxPooling2  (None, 14, 14, 64)        0         
 D)                                                              
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 12, 128)       73856     
                                                                 
 conv2d_3 (Conv2D)           (None, 10, 10, 128)       147584    
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 5, 5, 128)         0         
 g2D)                                                   

In [5]:
model.compile( optimizer= Adam(learning_rate=0.0001), loss = "categorical_crossentropy", metrics=['accuracy'])

In [6]:
es = EarlyStopping(
                    monitor='loss',
                    patience=2,
                    min_delta=0.01
                   )

In [7]:
model.fit(train_generator, epochs = 50, steps_per_epoch = 128, verbose = 1, validation_steps = 3, validation_data=validation_generator,  callbacks = [es])

Epoch 1/50
128/128 [==============================] - 64s 490ms/step - loss: 2.1555 - accuracy: 0.2068 - val_loss: 1.7592 - val_accuracy: 0.3567
Epoch 2/50
128/128 [==============================] - 55s 431ms/step - loss: 1.4411 - accuracy: 0.5020 - val_loss: 1.1066 - val_accuracy: 0.6433
Epoch 3/50
128/128 [==============================] - 56s 439ms/step - loss: 0.9171 - accuracy: 0.6994 - val_loss: 0.6303 - val_accuracy: 0.8267
Epoch 4/50
128/128 [==============================] - 57s 448ms/step - loss: 0.6216 - accuracy: 0.8073 - val_loss: 0.5061 - val_accuracy: 0.8600
Epoch 5/50
128/128 [==============================] - 47s 369ms/step - loss: 0.4695 - accuracy: 0.8592 - val_loss: 0.2808 - val_accuracy: 0.9200
Epoch 6/50
128/128 [==============================] - 49s 385ms/step - loss: 0.3659 - accuracy: 0.8909 - val_loss: 0.2062 - val_accuracy: 0.9433
Epoch 7/50
128/128 [==============================] - 46s 362ms/step - loss: 0.3042 - accuracy: 0.9095 - val_loss: 0.2405 - val_ac

In [8]:
predictions = model.predict(test_generator)

798/798 [==============================] - 269s 337ms/step


In [9]:
prediction = np.argmax(predictions,axis=1)

In [12]:
np.savetxt("prediction_one_hot_encoding.csv", X=predictions, delimiter=" ")

In [15]:
np.savetxt("prediction.csv", X=prediction, fmt='%i')